In [67]:
# Importing packages
import numpy as np
import pandas as pd
import itertools
import statistics as stat
import time

In [68]:
# Loading in CSV data.  For now, this only contains a single class' cards and initiatives
rawdata = pd.read_csv("GloomhavenTest.csv")
gen_classinfo = pd.read_csv("ClassInfo.csv")

In [69]:
# Setting data items for use later in code
# Will want to make the hand size set dynamic and then build a loop to cycle through current levels

active_class = gen_classinfo['Class'][0]
hand_size = gen_classinfo['Hand Size'][0]      # Setting the hand size
current_level = 1   # Current level for the loop (will likely build this loop out in VSC)

In [70]:
# Converting the level and initiative column to lists
level_df = rawdata[(rawdata['Class'] == active_class) & (rawdata['Level'] <= current_level)]['Level']
initiative_df = rawdata[(rawdata['Class'] == active_class) & (rawdata['Level'] <= current_level)]['Initiative']

There are three validation checks we need to do on each hand combination that was passed into the previous array in order to evaluate if it's a legitimate hand or not:

1.  The hand level sum needs to be lower than the max value for the hand and level
    - This is done by assigning each card in the combination it's level, so a level 1 card gets a 1, level 2 a 2, etc.
    - The sum of the array's level values then needs to be lower than the max value given by the following formula:
        - Max value formula = ((([1] + [Level])*[Level])/2) + ([Hand Size] - [Level])
        - Ex: For the Brute class, when evaluating hand combinations at level 2's selection time:
            - (((1+2)*2)/2) + (10-2)
            - 3 + 8 = 11
        - Intuitively, this makes sense as at level 1, the Brute class would have 10 level 1 cards for a total of 10
        - At level 2, at most, you would select one level 2 card and replace a level 1 card with it
        - That would replace a 1 value with a 2, taking the max value possible up to 11
        - If we ever run into a class that has a hand size lower than 8, this formula will need to be tweaked
            - This is because at a hand size of 8, you could remove a level 1 card for each level up
            - At a hand size of 7 or lower, the top level ups would mean you need to replace a card value over 1
        
2. At the level being evaluated, each hand combination can only have one of the level cards
    - When leveling up, you can initially only select a single card for the given level
    - This will filter out any combinations that load in something like two level 2 cards when evaluating level 2
    
3. The amount of cards over level 1 in the hand cannot exceed then evaluation level minus 1
    - This is because you can only make one card selection every time you level up
    - It is allowed for you to take a card lower than the level you're on, for example taking the 2nd lvl 2 card at level 3's promotion rather than a level 3
    - You cannot though have something like both level 2 cards AND a level 3 card at level 3
    - As the levels get higher, there are ways to make summation combinations that fall meet conditions 1 and 2 but use more cards > level 1 than is permitted

In [71]:
# Creating a max value function that will evaluate the value for condition 1 above
def maxvalue(handsize, current_level):
    return (((1 + current_level)*current_level)/2) + (handsize - current_level)

# Creating a function to count the number of cards at a level in a list
def lvl_counter(input_list, value):
    lvl_count = 0
    for i in input_list:
        if i == value:
            lvl_count += 1
    return lvl_count

In [72]:
# Creating a list of all the possible combinations.  This will get parsed in the following steps
level_comb = list(itertools.combinations(level_df, hand_size))
initiative_comb = list(itertools.combinations(initiative_df, hand_size))

In [73]:
# Performing validation checks on array and inputing hands that pass checks into a new list
valid_hand_array = list()
max_value = maxvalue(hand_size, current_level)

for ele, hand in enumerate(level_comb):
    if sum(hand) <= max_value and \
    (hand_size - lvl_counter(hand,1)) <= (current_level - 1) and (lvl_counter(hand, current_level)) != 2:
        
        valid_hand_array.append(ele)

In [74]:
# Performing validation checks on array and inputing hands that pass checks into a new list
valid_initiative_array = list()
hand_avg_init = list()

for ele, hand in enumerate(initiative_comb):
    if ele in valid_hand_array:
        
        valid_initiative_array.append(hand)
        hand_avg_init.append(sum(hand)/hand_size)

In [75]:
hand_describe = pd.DataFrame(hand_avg_init)
print(hand_describe.describe())

                0
count  286.000000
mean    44.000000
std      3.995919
min     33.600000
25%     41.125000
50%     44.200000
75%     46.675000
max     52.900000
